## [ESPM-136] Notebook 2: Company Emissions Exploratory Data Analysis (EDA)

Welcome to the second Jupyter Notebook for ESPM 136! In the previous notebook, you went through a small introduction to coding and performing data analysis in Python and Jupyter Notebooks. In this notebook, we'll be going through a **more applicable data analysis and visualization process of real-world data**, created straight from the company emissions Google Form you filled out before!

The process of looking at, summarizing, analyzing, and visualizing data that we'll be going through today is commonly referred to as **exploratory data analysis**. As we go through the notebook, hopefully you'll understand just how useful it can be for gaining a better understanding of trends in data we're interested in.

## Feeling stuck or want extra help with this notebook? Contact Data Peer Consultants!

If you find yourself having trouble with any content in this notebook, **Data Peer Consultants** are an excellent resource! Click [here](https://dlab.berkeley.edu/training/frontdesk-info) to locate live help.
Peer Consultants are there to answer all data-related questions, whether it be about the content of this notebook, applications of data science in the world, or other data science courses offered at Berkeley.

<hr style="border: 2px solid #003262">
<hr style="border: 2px solid #C9B676">

## Learning Outcomes

Working through this notebook, you will learn about:
1. Basic table functions for **viewing and conditionally selecting data**
2. More advanced table functions for **performing detailed data analysis**
3. **Creating visualizations** to better understand our data

<hr style="border: 2px solid #003262">
<hr style="border: 2px solid #C9B676">

## Recap of What We've Done So Far

In Assignment #1 you **gathered carbon emissions data and evaluated internal decarbonization strategies for two companies of your choice** and submitted all of this information into a Google Form. We will now be **using Jupyter Notebooks to analyze the data** that you collected!

As you learned in the first notebook, we use Jupyter Notebooks because this interactive environment allows us to work with code, text, and visualizations all at once. This is important especially in the context of corporate decarbonization because we want to learn how to manipulate the data you collected, as well as understand the meaning behind the manipulations to **understand how a company's carbon reporting compares with the actual internal actions they are taking to decarbonize.**

<hr style="border: 2px solid #003262">
<hr style="border: 2px solid #C9B676">

## Importing Modules and Taking an Initial Look at the Data

As stated previously, in this notebook we'll be utilizing data exported from the Google Form that you filled out based on company emissions. From the Google Form, we were able to export the results as a CSV (comma-separated value) file. We cleaned up and anonymized the data, and we uploaded it to the same folder that this notebook is located in. Thanks to this pre-processing, we can now load in our data to a table object from the `datascience` module!

As with the first notebook, we need to import the modules necessary for this notebook. You can do this by running (clicking the cell and hitting `Cmd/Ctrl`+`Enter`, or clicking the cell and hitting the `Run` button at the top) the following code cell!

In [1]:
# Run this cell to import the modules
import numpy as np
import pandas as pd
from datascience import *
from ipywidgets import *
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
import seaborn as sns, plotly.express as px

Now that we have all the modules we need, we can load our original CSV file into a table object from the `datascience` module!

The `.show()` command in the cell below allows us to see a specified number of rows of the table. In this case, we want to see the first five rows of the table to see what it looks like.

In [2]:
# Run this cell to create our emissions table object
emissions = Table.read_table('emissions.csv')
emissions.show(5)

Index,Company Name,Company Sector,Company Sector (Other),Year of CDP Disclosure,Scope 1 value,Scope 2 value (location-based),Scope 2 value (market-based),Scope 3 value,Total Revenue,Currency of Total Revenue,Internal Price on Carbon (Y/N),Price
0,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85
1,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85
2,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85
3,AHOLD DELHAIZE,CONSUMER PACKAGED GOODS,GROCERY / FOOD DISTRIBUTION,2022,1.728e+06,1.748e+06,1.099e+06,6.59304e+07,75600,EUR,Yes,150
4,AIR FRANCE-KLM,TRANSPORTATION,nan,2022,1.63363e+07,nan,19104.7,8.70067e+06,1.4315e+10,EUR,Yes,nan


In the view of our table above, particularly in the `Company Sector (Other)` and `Scope 2 value (location-based)` columns, you may see multiple values called `nan`. In Python, missing or undefined values are represented by `nan` values, which stands for "Not a Number". You can think of these as missing or N/A values!

Another small note on notation that you may notice in the table above and throughout this notebook: in some of the numerical columns, you may see the number followed by a value like `e+06` or `e+10`. This notation is Python's way of displaying very large or very small numbers. `e+` in this case means a positive power of 10 (`e-` would be a negative power, which would be a way of representing very small decimal numbers). So, when you see a number like `e+06`, this means $10^6$, which is a million. In the table above, `7.25451e+06` means $7.25451 * 10^6$, which is actually equal to $7,254,510$. This notation is not the easiest for us to read, but it is a very useful method for Python to store large and small numbers simply without losing any information!

As we saw in Notebook 1, we can use the `.num_rows` and `.num_columns` methods to take a look at the dimensions of our table.

In [3]:
emissions.num_rows

156

In [4]:
emissions.num_columns

13

We have 13 different columns and 156 rows. Each column corresponds to a feature of our data point, as described in further detail below. Each row corresponds to one company of the two that were submitted by each student via the Google Form. The total number of rows ended up being 156 after we removed a few duplicate submissions to the Google Form.

The following code cell is a bit more advanced, but we can use it to show us the number of distinct companies that were reported in our table.

In [5]:
len(emissions.group('Company Name')['count'])

103

## Data Dictionary

Now that we've taken an intial glance at the data that you collected, let's familiarize ourselves with the contents of the `emissions` table. The **data dictionary** below reveals information about each column in the table (excluding the `Index` column, which just provides a number for each row) regarding what the column represents and the type of the data in the column. Data dictionaries are useful for providing context and understanding of what the data represents for anyone working with the data.

|Column Name| Meaning |Type|
|--|--|--|
|Company Name| The name of the company chosen to report data| category |
|Company Sector| The industry sector the company belongs in| category |
|Company Sector (Other)|An optional field if the company's sector was not one of the given options|category |
|Year of CDP Disclosure|The submission year of the report. The data provided represents the previous year.|number|
|Scope 1 value|The company's direct GHG emissions, given in metric tons of CO2e|number|
|Scope 2 value (location-based)|The company's indirect (location-based) GHG emissions, given in metric tons of CO2e |number |
|Scope 2 value (market-based)|The company's indirect (market-based) GHG emissions, given in metric tons of CO2e|number |
|Scope 3 value| The company's indirect GHG emissions that come from its value chain, given in metric tons of CO2e| number |
|Total Revenue| The company's total revenue for a given year| number |
|Currency of Total Revenue|The currency of the company's total revenue|category|
|Internal Price on Carbon|An indicator of if the company implements internal carbon pricing |category|
|Price|Internal price on carbon  | number|

<hr style="border: 2px solid #003262">
<hr style="border: 2px solid #C9B676">

## 1. Basic Table Operations: `select`, `where`, `sort`

Now that we have learned how to import our data from a CSV file, we can begin doing some interesting exploratory data analysis to see what information our data can tell us! Oftentimes, our data has a lot more information than we actually need, or it may not be in the correct order. To obtain the data we want, we can use utilize basic table methods such as `select`, `where`, and `sort` to filter out columns, rows, and sort our data.

### 1.1 Selecting Columns

Sometimes when doing data analysis, we only want to look at a subset of columns. Having too many columns can make the table hard to read if we are only interested in certain information. **We can pick which columns we want to see using the table method `.select()`.**

When we call this method, we specify which columns we want, and it returns a brand new table with only the specified columns in the order they are inputted.

For example, if we wanted to select only the companies and their sector we can run the code below.

In [6]:
emissions.select('Company Name', 'Company Sector')

Company Name,Company Sector
ADIDAS,APPAREL & FOOTWEAR
ADIDAS,APPAREL & FOOTWEAR
ADIDAS,APPAREL & FOOTWEAR
AHOLD DELHAIZE,CONSUMER PACKAGED GOODS
AIR FRANCE-KLM,TRANSPORTATION
AIRBUS,OTHER
ALBERTSONS COMPANIES,CONSUMER PACKAGED GOODS
ALLY FINANCIAL,OTHER
ALSTOM,OTHER
AMCOR,CONSUMER PACKAGED GOODS


### 1.2 Conditioning on Rows

We now know how to select certain columns of interest, but what if we wanted to only see rows in which the data meets a certain condition? The `.where()` method allows us to do exactly this. 

This method takes in two arguments:
   1. The **name of the column** whose values must meet a certain condition
   2. The predicate function that states **the condition the row must meet**
   
For example, if we wanted to only select the rows where the company is H&M, we could do the following:

In [7]:
emissions.where('Company Name', are.equal_to('H&M'))

Index,Company Name,Company Sector,Company Sector (Other),Year of CDP Disclosure,Scope 1 value,Scope 2 value (location-based),Scope 2 value (market-based),Scope 3 value,Total Revenue,Currency of Total Revenue,Internal Price on Carbon (Y/N),Price
54,H&M,APPAREL & FOOTWEAR,nan,2022,21171,507170,37394,7.74163e+06,1.99e+11,SEK,Yes,0
55,H&M,APPAREL & FOOTWEAR,nan,2022,21171,507170,37394,7.74163e+06,1.99e+11,SEK,Yes,0
56,H&M,APPAREL & FOOTWEAR,nan,2022,21171,507170,37394,7.74163e+06,1.99e+11,SEK,Yes,0
57,H&M,APPAREL & FOOTWEAR,nan,2022,21171,507170,37394,7.74163e+06,1.99e+11,SEK,Yes,nan
58,H&M,APPAREL & FOOTWEAR,nan,2022,21171,507170,37394,5.43711e+06,1.99e+11,SEK,Yes,0
59,H&M,APPAREL & FOOTWEAR,nan,2022,21171,507170,37394,7.74163e+06,1.99e+11,SEK,Yes,0
60,H&M,APPAREL & FOOTWEAR,nan,2022,21171,507170,37394,5.43711e+06,1.99e+11,SEK,Yes,0
61,H&M,APPAREL & FOOTWEAR,nan,2022,21171,507170,37394,5.43711e+06,1.99e+11,SEK,Yes,0
62,H&M,APPAREL & FOOTWEAR,nan,2022,21171,507170,37394,7.73413e+06,1.99e+11,SEK,Yes,nan
64,H&M,APPAREL & FOOTWEAR,nan,2021,21171,507170,37394,2.30452e+06,2.3e+10,USD,Yes,0


The cell above outputs a copy of the original table but **only includes rows where the company is H&M**. There are many other predicate functions -- here are a few common ones you may find useful in the next notebook: 

|Predicate|Example|Result|
|-|-|-|
|`are.equal_to`|`are.equal_to(50)`|Find rows with values equal to 50|
|`are.not_equal_to`|`are.not_equal_to(50)`|Find rows with values not equal to 50|
|`are.above`|`are.above(50)`|Find rows with values above (and not equal to) 50|
|`are.above_or_equal_to`|`are.above_or_equal_to(50)`|Find rows with values above 50 or equal to 50|
|`are.below`|`are.below(50)`|Find rows with values below 50|
|`are.between`|`are.between(2, 10)`|Find rows with values above or equal to 2 and below 10|
|`are.between_or_equal_to`|`are.between_or_equal_to(2, 10)`|Find rows with values above or equal to 2 and below or equal to 10|

### 1.3 Sorting Values

Aside from selecting the desired columns and rows that we want, we can also sort the values in a certain column. The `.sort()` method accomplishes this if we pass in the column we want to sort by. By default, this method sorts a column's values in ascending order (in other words, we see the smallest values first) unless an *optional* second argument `descending = True` is passed into `.sort()`.

If we wanted to sort a column that contains text, `.sort()` will sort the values alphabetically from A-Z (i.e. ascending order). For example: 

In [8]:
emissions.sort('Company Sector')

Index,Company Name,Company Sector,Company Sector (Other),Year of CDP Disclosure,Scope 1 value,Scope 2 value (location-based),Scope 2 value (market-based),Scope 3 value,Total Revenue,Currency of Total Revenue,Internal Price on Carbon (Y/N),Price
0,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85
1,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85
2,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85
12,AMERICAN EAGLE OUTFITTERS,APPAREL & FOOTWEAR,nan,2022,9248,53642,40231,1.4793e+06,5.01078e+09,USD,No,nan
13,AMERICAN EAGLE OUTFITTERS,APPAREL & FOOTWEAR,nan,2022,9248,53642,40231,3.6639e+06,5.01078e+09,USD,No,nan
16,ARITZIA,APPAREL & FOOTWEAR,nan,2022,1560,2936,nan,446097,1.49463e+09,CAD,No,nan
27,BURBERRY,APPAREL & FOOTWEAR,nan,2022,1834,25866,nan,399973,2.826e+09,GBP,Yes,12.87
46,FILA HOLDINGS,APPAREL & FOOTWEAR,nan,2022,1245.9,3005.5,nan,0,3.794e+12,KRW,No,nan
53,GIORGIO ARMANI,APPAREL & FOOTWEAR,nan,2022,4263,21614,15306,353442,2.05558e+09,EUR,No,nan
54,H&M,APPAREL & FOOTWEAR,nan,2022,21171,507170,37394,7.74163e+06,1.99e+11,SEK,Yes,0


Our `emissions` data is already sorted by company name alphabetically from A-Z. However, if we wanted to sort the company names alphabetically from Z-A (i.e. descending order), we would run the cell below.

In [9]:
emissions.sort('Company Name', descending = True)

Index,Company Name,Company Sector,Company Sector (Other),Year of CDP Disclosure,Scope 1 value,Scope 2 value (location-based),Scope 2 value (market-based),Scope 3 value,Total Revenue,Currency of Total Revenue,Internal Price on Carbon (Y/N),Price
155,WALMART,CONSUMER PACKAGED GOODS,nan,2022,7.31224e+06,9.47517e+06,6.54417e+06,2.13846e+08,5.72754e+11,USD,No,nan
154,VOLKSWAGEN,OTHER,AUTOMOTIVE,2022,4.74087e+06,5.19596e+06,2.42214e+06,3.53561e+09,2.502e+11,EUR,Yes,475
153,VF CORPORATION,APPAREL & FOOTWEAR,nan,2022,12551,69212,54693,3.37573e+06,1.18418e+10,USD,No,0
152,VESTAS,ENERGY,nan,2022,99290,78640,2673,9.6961e+06,1.5587e+10,EUR,Yes,120
151,VALERO,ENERGY,nan,2022,2.713e+07,nan,4.88e+06,0,1.76382e+11,USD,No,nan
150,UPS (UNITED PARCEL SERVICE),TRANSPORTATION,nan,2022,1.5668e+07,676000,654000,1.9486e+07,9.6223e+10,USD,No,nan
149,UPFIELD,FOOD & AGRICULTURE,nan,2022,36059,77879,27798,2.96321e+06,2.793e+09,EUR,No,nan
148,UNILEVER,CONSUMER PACKAGED GOODS,nan,2022,565988,1.2445e+06,144752,6.14457e+07,5.2444e+10,EUR,Yes,70
147,ULTA BEAUTY,OTHER,COSMETICS,2022,6815,114295,107611,1.92119e+06,6.152e+09,USD,No,nan
144,TYSON FOODS,FOOD & AGRICULTURE,nan,2022,3.82629e+06,nan,nan,0,4.705e+10,USD,No,nan


From this section, hopefully you can see how basic table operations like `select`, `where`, and `sort` can be very useful when exploring our data!

<hr style="border: 2px solid #003262">
<hr style="border: 2px solid #C9B676">

## 2. More Advanced Table Operations: `apply`, `group`, `pivot`

Now that we have learned how to get the data we want from the dataset and format it to our preferences, we can start exploring more advanced table techniques such as applying functions to data, grouping data based on specific criteria, and pivoting data to reshape it for advanced analysis and visualization.

### 2.1 Apply

<div class="alert alert-block alert-success">
    <p style="font-size:20px">This section is advanced/optional
</div>

The `.apply()` method applies a function to each element of a data structure and returns the result of that function call for each element. When we call this function, we need to specify **the function we are using** and **which column(s) we are applying it to**.

For our data, a use case of the `.apply()` method would be performing the same calculation on each element of a column. For example, we notice from the dataset that the `Currency of Total Revenue` column contains different currencies, which could make numerical comparisons on the `Total Revenue` value difficult. As a result, we would like to implement a function on the `Total Revenue` column that changes all the currency to USD.

**Note:** A lot of the code in the following example is more advanced than we're expecting you to understand -- this just serves as an example of a more complex way in which we can use the `.apply()` method!

In [10]:
# This code helps us see all of the unique currencies in our dataset
np.unique(emissions['Currency of Total Revenue'])

array(['AUD', 'CAD', 'CHF', 'DKK', 'EUR', 'GBP', 'JPY', 'KRW', 'NOK',
       'RUB', 'SEK', 'USD'],
      dtype='<U3')

In [11]:
# Then, let's create a new column with both the revenue and its currency in the format of [Currency, Revenue]
combined = [[emissions['Currency of Total Revenue'][i], emissions['Total Revenue'][i]] for i in range(len(emissions['Currency of Total Revenue']))]
emissions['Revenue with Currency'] = combined
emissions.show(5)

Index,Company Name,Company Sector,Company Sector (Other),Year of CDP Disclosure,Scope 1 value,Scope 2 value (location-based),Scope 2 value (market-based),Scope 3 value,Total Revenue,Currency of Total Revenue,Internal Price on Carbon (Y/N),Price,Revenue with Currency
0,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85,['EUR' 21234000000.0]
1,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85,['EUR' 21234000000.0]
2,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85,['EUR' 21234000000.0]
3,AHOLD DELHAIZE,CONSUMER PACKAGED GOODS,GROCERY / FOOD DISTRIBUTION,2022,1.728e+06,1.748e+06,1.099e+06,6.59304e+07,75600,EUR,Yes,150,['EUR' 75600.0]
4,AIR FRANCE-KLM,TRANSPORTATION,nan,2022,1.63363e+07,nan,19104.7,8.70067e+06,1.4315e+10,EUR,Yes,nan,['EUR' 14315000000.0]


In the cell below, we write our own custom function that we'll be using with `.apply()`. Essentially, the function takes in a currency measurement and then returns the revenue value multiplied by the USD conversion rate.

Again, feel free to look through the code below to see how it works, but don't worry if you don't fully understand it!

In [12]:
# This function takes in all elements in the "Revenue with Currency" column and turns all the non-USD revenue into USD currency
def toUSD(i):
    dic = {
    'AUD': 0.75,   # 1 AUD to USD
    'CAD': 0.78,   # 1 CAD to USD
    'CHF': 1.09,   # 1 CHF to USD
    'DKK': 0.15,   # 1 DKK to USD
    'EUR': 1.16,   # 1 EUR to USD
    'GBP': 1.37,   # 1 GBP to USD
    'JPY': 0.009,  # 1 JPY to USD
    'KRW': 0.0009, # 1 KRW to USD
    'NOK': 0.11,   # 1 NOK to USD
    'RUB': 0.014,  # 1 RUB to USD
    'SEK': 0.11,   # 1 SEK to USD
    'USD': 1       # No Change
    }
    return i[1] * dic[i[0]] # The revenue multiplied by the corresponding USD conversion rate

In the cell below, we'll use `.apply()` to create a new column of the function applied to the currency column in order to receive the currencies in USD!

In [13]:
emissions['Revenue (USD)'] = emissions.apply(toUSD, 'Revenue with Currency')
emissions = emissions.drop('Revenue with Currency') # dropping this column so the dataset does not look too messy
emissions.show(5)

Index,Company Name,Company Sector,Company Sector (Other),Year of CDP Disclosure,Scope 1 value,Scope 2 value (location-based),Scope 2 value (market-based),Scope 3 value,Total Revenue,Currency of Total Revenue,Internal Price on Carbon (Y/N),Price,Revenue (USD)
0,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85,2.46314e+10
1,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85,2.46314e+10
2,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85,2.46314e+10
3,AHOLD DELHAIZE,CONSUMER PACKAGED GOODS,GROCERY / FOOD DISTRIBUTION,2022,1.728e+06,1.748e+06,1.099e+06,6.59304e+07,75600,EUR,Yes,150,87696
4,AIR FRANCE-KLM,TRANSPORTATION,nan,2022,1.63363e+07,nan,19104.7,8.70067e+06,1.4315e+10,EUR,Yes,nan,1.66054e+10


Here's another example of using `.apply()`: we can use it to transform a numerical variable into a categorical variable. As an example, say we want to categorize the `Total Revenue` amounts into "high", "medium", or "low". We can achieve this using the `.apply()` function!

In [14]:
# Run this cell to create the function needed for this operation
def rev_level(revenue):
    if revenue > 10 ** 10:
        return 'High'
    if revenue < 10 ** 5:
        return 'Low'
    else:
        return 'Medium'

In the following cell, we use the `.apply()` function in order to create a new column in our table called `Revenue Level`. If you don't see the new column after running the cell at first, be sure to scroll the table all the way to the right, as that is where new columns automatically get added!

In [15]:
# Run this cell to apply the function to the dataset and add the new column
emissions['Revenue Level'] = emissions.apply(rev_level, 'Revenue (USD)')
emissions

Index,Company Name,Company Sector,Company Sector (Other),Year of CDP Disclosure,Scope 1 value,Scope 2 value (location-based),Scope 2 value (market-based),Scope 3 value,Total Revenue,Currency of Total Revenue,Internal Price on Carbon (Y/N),Price,Revenue (USD),Revenue Level
0,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85,2.46314e+10,High
1,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85,2.46314e+10,High
2,ADIDAS,APPAREL & FOOTWEAR,nan,2022,12908.4,nan,125502,7.25451e+06,2.1234e+10,EUR,Yes,85,2.46314e+10,High
3,AHOLD DELHAIZE,CONSUMER PACKAGED GOODS,GROCERY / FOOD DISTRIBUTION,2022,1.728e+06,1.748e+06,1.099e+06,6.59304e+07,75600,EUR,Yes,150,87696,Low
4,AIR FRANCE-KLM,TRANSPORTATION,nan,2022,1.63363e+07,nan,19104.7,8.70067e+06,1.4315e+10,EUR,Yes,nan,1.66054e+10,High
5,AIRBUS,OTHER,ROLLING STOCK MANUFACTURER,2022,561737,339313,265016,4.76092e+08,52856,EUR,Yes,150,61313,Low
6,ALBERTSONS COMPANIES,CONSUMER PACKAGED GOODS,GROCERY / FOOD DISTRIBUTION,2022,2.76738e+06,1.66352e+06,1.64609e+06,7.50445e+06,6.96904e+10,USD,No,nan,6.96904e+10,High
7,ALLY FINANCIAL,OTHER,FINANCE / BANKING,2022,4228,9758,0,13986,8.381e+09,USD,No,nan,8.381e+09,Medium
8,ALSTOM,OTHER,AEROSPACE,2022,107073,144632,121789,3.34293e+07,1.5741e+10,EUR,No,nan,1.82596e+10,High
9,AMCOR,CONSUMER PACKAGED GOODS,nan,2022,529192,1.43269e+06,1.43208e+06,1.96516e+07,1.2861e+10,USD,No,nan,1.2861e+10,High


Something to note here: being able to analyze the data by creating new columns or subsets of our data like this can also be useful for sanity checking our data. For example, in the output above, we can double check that our data is accurate by ensuring our companies are in the correct `Revenue Level` categories. From this analysis, we could notice any suspiciously small or large numbers, such as the example of `AHOLD DELHAIZE` above; these intermediate steps give us a chance to check the accuracy of our data early on in our analysis!

These examples were more advanced but applicable use cases of the `.apply()` function. It can also be used in simpler cases; as an example, if we had data in a column that needed to be multiplied or divided by a specific factor, we could easily use `.apply()` to apply the multiplication / division to every piece of data in the column!

### 2.2 Group

Since we now have revenues with the same format, we can perform some aggregation of our data with the `.group()` function, which  groups data by a specific attribute, allowing for analysis and numeric summarization within each group. In the following example, we can use it to look at the average revenue of each company sector.

In the cell below, you can see the call to `.group()` in the second line, **where the first value in the parenthesis is the column name we want to group by, and the second value in the parenthesis is the function we want to use to aggregate the numeric data together** (in this case, we're calculating the average / mean revenue in USD of the data grouped by company sector).

In [16]:
emission_g = emissions.select('Company Sector', 'Revenue (USD)') # Select the columns we want to look at
emission_g.group('Company Sector', np.mean) # Call the group function with the column and aggregation function we want to use

Company Sector,Revenue (USD) mean
APPAREL & FOOTWEAR,1.21045e+10
CONSUMER PACKAGED GOODS,6.82165e+10
ENERGY,8.53938e+10
FOOD & AGRICULTURE,2.68682e+10
MATERIALS,2.90295e+10
OTHER,2.47897e+10
TECHNOLOGY,9.02242e+10
TRANSPORTATION,3.60605e+10


In the next cell, we use another aggregation function (the median) to show that there are multiple aggregation functions we can use to summarize our data.

In [17]:
# We could use other numeric functions besides the average, such as the median, with our grouped data
emission_g.group('Company Sector', np.median)

Company Sector,Revenue (USD) median
APPAREL & FOOTWEAR,5.38739e+09
CONSUMER PACKAGED GOODS,1.29282e+10
ENERGY,2.90683e+10
FOOD & AGRICULTURE,1.743e+10
MATERIALS,2.90295e+10
OTHER,6.152e+09
TECHNOLOGY,8.08485e+10
TRANSPORTATION,1.27989e+10


The `.group()` method also allows us to group the data by more than one variable! In the following cell, we put the `Company Name` and `Company Sector` column names in a list (denoted by the square brackets) and then group by those values. From there, we calculate the maximum value of the revenue in USD among these groupings!

In [18]:
# We can also group the data by more than one variable
emission_g1 = emissions.select('Company Sector', 'Company Name', 'Revenue (USD)')
emission_g1.group(['Company Sector', 'Company Name'], np.max)

Company Sector,Company Name,Revenue (USD) max
APPAREL & FOOTWEAR,ADIDAS,2.46314e+10
APPAREL & FOOTWEAR,AMERICAN EAGLE OUTFITTERS,5.01078e+09
APPAREL & FOOTWEAR,ARITZIA,1.16581e+09
APPAREL & FOOTWEAR,BURBERRY,3.87162e+09
APPAREL & FOOTWEAR,FILA HOLDINGS,3.4146e+09
APPAREL & FOOTWEAR,GIORGIO ARMANI,2.38447e+09
APPAREL & FOOTWEAR,H&M,2.3e+10
APPAREL & FOOTWEAR,INDITEX,3.21692e+10
APPAREL & FOOTWEAR,LEVI STRAUSS & CO.,5.764e+09
APPAREL & FOOTWEAR,LYMI DBA REFORMATION,1.71381e+06


### 2.3 Pivot

<div class="alert alert-block alert-success">
    <p style="font-size:20px">This section is advanced/optional
</div>

The `.pivot()` function in Python reshapes data, essentially turning the rows into columns. Similar to `.group()`, it summarizes data, but it is a little more complicated as it changes column values in our original table into column indexes in a new table. 

For example, if we want to see the count of each currency in each company sector, we can use the `.pivot()` function as shown below:

In [19]:
emissions.pivot('Currency of Total Revenue', # the row index
                'Company Sector', # the column index
                )

Company Sector,AUD,CAD,CHF,DKK,EUR,GBP,JPY,KRW,NOK,RUB,SEK,USD
APPAREL & FOOTWEAR,0,2,0,0,8,1,0,1,0,0,9,23
CONSUMER PACKAGED GOODS,0,0,0,0,5,0,0,0,0,0,0,8
ENERGY,0,0,0,4,6,0,1,0,1,1,0,21
FOOD & AGRICULTURE,1,0,3,0,2,1,0,0,0,0,0,21
MATERIALS,0,0,0,0,0,0,0,0,0,0,0,2
OTHER,0,0,0,0,3,0,1,2,0,0,0,13
TECHNOLOGY,0,0,0,0,0,0,0,0,0,0,0,4
TRANSPORTATION,0,0,0,0,4,0,1,0,0,0,0,7


Instead of value counts, we can also incorporate aggregations into this function just like we did with `.group()`.

Now, let's look at the average `Scope 1 value` for each company sector, dividing the data based on whether they have an internal price on carbon or not.

In [5]:
emissions.pivot('Internal Price on Carbon (Y/N)', # the row index
                'Company Sector', # the column index
                values = 'Scope 1 value', # the variable that is being evaluated for each combination
                collect = np.mean # how the value is going to be aggregated
                )

Company Sector,No,Yes
APPAREL & FOOTWEAR,7225.83,16374.2
CONSUMER PACKAGED GOODS,1.59912e+06,1.44563e+06
ENERGY,1.02211e+07,2.59892e+07
FOOD & AGRICULTURE,1.72645e+06,2.79406e+06
MATERIALS,0,7.64138e+07
OTHER,1.93513e+06,1.34584e+07
TECHNOLOGY,66042.1,0
TRANSPORTATION,9.34322e+06,6.3131e+06


This pivot table shows us the average `Scope 1 value` for each combination of `Company Sector` and whether they have an internal price on carbon or not! For example, in the first row, we are able to clearly see the two average `Scope 1 value`s for the `APPAREL & FOOTWEAR` sector, broken down by whether the companies have an internal price on carbon. It allows us to better visualize how the average `Scope 1 value` changes based on having an internal price on carbon or not. We can look at the two values for each sector and consider whether the values are what we expect or not as another form of exploration.

**Note:** The reason we have two values of 0 in the pivot table above is not because 0 is the average `Scope 1 value`, but because we didn't have any values that were combinations of the given row and column (for example, we had no rows in our dataset from the `MATERIALS` sector that did not have an internal price on carbon).

From this, we can see how pivot tables allow us to more clearly focus on numerical trends that we're interested in.

<hr style="border: 2px solid #003262">
<hr style="border: 2px solid #C9B676">

## 3. Visualizations

In this section, we'll be able to **analyze our data visually by creating various charts and graphs.** We'll start off by looking at basic charts that we can create with the built-in visualizations with the `datascience` module. Then, we'll show off some more advanced visualizations that we can create with other visualization modules!

**For Section 3.2 on advanced visualizations, don't worry too much about understanding how to write the code!** We won't ask you to re-create those types of graphs in the next notebook; instead, just focus on analyzing the graph visually to see what information you can gather from it.

### 3.1 Basic Scatter Plots and Bar Charts with the `datascience` Module

Utilizing the `datascience` module and its built-in graphing functions, let's create some basic scatter plots and bar charts that we learned about in Notebook 1.

First, let's practice creating a scatter plot in order to look at the relationship between two numeric values. For the following graph, we use `.scatter()` to view the relationship between the `Scope 1 value` and `Scope 2 value`.

In [ ]:
emissions.scatter('Scope 1 value', 'Scope 2 value (market-based)')

From the x-axis and y-axis above, we can see that the `Scope 2 value` has a much larger range of values it takes on than the `Scope 1 value`. We can also see that many of the points are concentrated in the bottom left corner, and we only have a handful of **outlier data points** that have larger Scope 1 and/or Scope 2 values.

Next, let's look at some examples of bar charts. The code for these bar charts is a bit more complex, because we need to take some intermediate steps of grouping, sorting, and selecting certain columns and rows of the table in order to get a more readable graph.

**If the code is a little confusing, that's okay! Just focus on being able to interpret what the graphs are telling us.**

For the first bar chart, we group our data based on company sector and then look at the average `Scope 1 value` for every sector. From this, we can see that the `Materials` and `Energy` sectors have the highest average Scope 1 values.

In [ ]:
emissions.group('Company Sector', np.average).barh('Company Sector', 'Scope 1 value average')

In the following bar chart, we want to look at the top five companies based on the average of their `Scope 1 value`. 

To accomplish this, we first have to group the data by the company (in case multiple students submitted data on the same company), and then sort the data in descending order of the average `Scope 1 value`. From this, we take the top 5 companies and create a bar chart from it.

We've broken up this code into intermediate steps below so that it is more readable, but all of this code could have been chained together into one long line!

In [ ]:
grouped_company = emissions.group('Company Name', np.average)
sorted_grouped_company = grouped_company.sort('Scope 1 value average', descending = True)
top_five_companies = sorted_grouped_company.take([0, 1, 2, 3, 4])

# In the following line, we create the bar chart: categorical column on the y-axis, numeric column on the x-axis
top_five_companies.barh('Company Name', 'Scope 1 value average')

### 3.2 More Advanced Visualizations

The `datascience` module's graphing methods are quite beginner-friendly, but if we want to create more advanced, professional-looking, or informative graphs, we can use other data visualization modules, such as `Seaborn` and `Plotly`!

**You won't be expected to know how to write the code to generate the following graphs for the next notebook;** we're just taking a look at them to see how we can create more complex graphs.

For example, we can use the `Seaborn` module (which we imported at the very beginning as `sns`) to create a graph of the counts of each company sector. From the graph below, we can see that the most common companies students picked to report data on were part of the `Apparel & Footwear` and `Energy` sectors!

In [ ]:
sns.countplot(y = emissions['Company Sector'])
plt.show()

Next up, using the `Plotly module`, we can get really fancy with our graphs! The following graph is an interactive one -- once you run it, you'll have the option to select a type of `Currency`. This option selects only companies with that type of currency (so for the less common currency types, you'll only see a few points plotted).

It then creates a scatterplot of the company's `Total Revenue` plotted against their `Scope 1 value`. It also colors the company's point based on the sector it belongs to. Select different options for the currency and see what you find!

**Note:** This plot may take a few seconds to load at first and each time you select a different currency.

In [ ]:
df = pd.read_csv('emissions.csv')

@interact(currency = Dropdown(options = np.unique(emissions['Currency of Total Revenue']), value = 'USD'))
def plot_px(currency):
    mask = df['Currency of Total Revenue'].isin([currency])
    fig = px.scatter(df[mask], x = 'Total Revenue',
                     y = 'Scope 1 value',
                     color = 'Company Sector',
                     hover_data = ['Company Name'],
                     labels= {
                        'Scope 1 value': 'Emissions',
                     },
                 )
    fig.update_layout(xaxis_title = "Total Revenue",
                      yaxis_title = "Total Direct GHG Emissions")
    fig.show()

Pretty cool, right? Let's see how we can utilize `Plotly` and these interactive graphs further by next focusing on one sector.

### 3.3 A Closer Look at the Food & Agriculture Sector

Let's take a closer look at one of the most popular sectors that your class looked at: Food & Agriculture. Below, data from 2018 through 2021 is shown for nine of the most prominent food and agricultural companies, ranging from beverage companies (Molson Coors, PepsiCo), food retailers (Tesco), and famous food companies (Nestlé). This data was gathered from a variety of sources, including various CDP reports, company sustainability reports, and the SEC (U.S. Securities and Exchange Commission) website.

In [ ]:
# Run this cell to load the data
food_ag = pd.read_csv('foodag.csv')
food_ag.head()

#### 3.3.1 GHG Emissions Over Time

Let's take a look at some trends in the total emissions of the companies over the years 2018 through 2021. Are the total emissions in the overall food & agriculture sector generally decreasing, increasing, staying flat, or fluctuating over time?

In [ ]:
companies = food_ag['COMPANY NAME'].unique()
years = ['2018', '2019', '2020', '2021']

@interact(x = widgets.Dropdown(options = list(companies), value = 'Danone'))
def g(x):
    emissions = food_ag[food_ag['COMPANY NAME'] == x]["TOTAL EMISSIONS"] / 1000
    bar_container = plt.bar(years, emissions);
    plt.ylim(0, max(emissions) + 5000)
    plt.xlabel('Year')
    plt.ylabel('Total Scope 1 & Scope 2 Emissions (thousand mt CO2e)', size = 11)
    plt.title(x + ' GHG Emissions (2018-2021)', size = 15)
    plt.bar_label(bar_container, fmt='{:,.0f}')
    plt.grid(False)

#### 3.3.2 Carbon Intensity

Another way we can make comparisons within a sector is by looking at a company's carbon intensity. **Carbon intensity is a measure of GHG emissions per unit of activity. For this data, carbon intensity is represented as the total scope 1 and scope 2 (location-based) emissions per unit of the company's revenue.**

Run the code cell below to observe trends in carbon intensity for each company in the food and agriculture sector over time.

In [ ]:
companies = food_ag['COMPANY NAME'].unique()
years = ['2018','2019', '2020', '2021']

for co in companies:
    ci = food_ag[food_ag['COMPANY NAME'] == co]["CARBON INTENSITY\n(Scope 1 & 2 g CO2e / $ Sales)"]
    plt.plot(years, ci, label = co)
plt.legend(loc = 'upper right', fontsize = 'xx-small')
plt.xlabel('Year')
plt.ylabel('Carbon Intensity')
plt.title('Food & Ag Carbon Intensity Over Time (2018-2021)');

We can also look at carbon intensity trends for each individual company. Run the cell below to observe different carbon intensity trends.

In [ ]:
@interact(x = widgets.Dropdown(options = list(companies), value = 'Danone'))
def g(x):
    ci = food_ag[food_ag['COMPANY NAME'] == x]["CARBON INTENSITY\n(Scope 1 & 2 g CO2e / $ Sales)"] 
    bar_container = plt.plot(years, ci);
    plt.xlabel('Year')
    plt.ylabel('Carbon Intensity (metric tons CO2e / mill total dollars)', size = 11)
    plt.title(x + ' Carbon Intensity (2018-2021)', size = 15)
    plt.grid(False)

Hopefully this section of visualizations has shown you just how much more we can understand from our data by creating various graphs! 

As a reminder, in the next notebook, **you'll only be expected to re-create graphs similar to those in Section 3.1.** We may ask you some written questions about graphs similar to the ones you saw in sections 3.2 and 3.3, but you aren't expected to know how to write the code for them!

<hr style="border: 2px solid #003262">
<hr style="border: 2px solid #C9B676">

## Conclusion

In this notebook, you've learned quite a bit, below is a summary of the topics we have covered:
- Process of our Data Collection and Importing Modules
    - Data Dictionary
    - Structure of a Notebook
- Basic Table Operations 
    - Select
    - Sort
    - Where (Conditioning)
- More Advanced Table Operations
    - Apply
    - Group
    - Pivot
- Visualizations
    - Basic data visualizations with `datascience`
    - More advanced visualizations with `Seaborn` and `Plotly`
    
In the next notebook, you'll be getting some hands-on experience with many of these methods of performing data analysis!

<h3>Congratulations on finishing the notebook!</h3>

<hr style="border: 2px solid #003262">
<hr style="border: 2px solid #C9B676">

## A Final Request: Feedback Form

<div class="alert alert-info">
<b> We encourage students to fill out the following feedback form to share your experience with this Modules notebook. This feedback form will take no longer than 5 minutes. At UC Berkeley Data Science Undergraduate Studies Modules, we appreciate all feedback to improve the learning of students and experience utilizing Jupyter Notebooks for Data Science Education: </b> 
</div>

# [UC Berkeley Data Science Feedback Form](https://forms.gle/hPgYMxFWKXH2sVkd7)